In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Libraries imported.


In [3]:
with open(r'C:\Users\USER\Desktop\0623_Aaron Final Project\newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [5]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [6]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [7]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [8]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [9]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [10]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'ZP5NRW0IDHT1BI55LYEJ0KG00HKDJJ2QYATE3NSC1KCEN1C5' # your Foursquare ID
CLIENT_SECRET = 'EMUFOL3QNNVOXV5HNLRJNAYNEINPSJWBARLBV55ORNNURHMF' # your Foursquare Secret
VERSION = '20181020'

In [12]:
#https://developer.foursquare.com/docs/resources/categories
#Sushi = 4bf58dd8d48988d1d2941735
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_bubbleTeaShop = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='52e81612bcbc57f1066b7a0c')
newyork_venues_bubbleTeaShop.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Chinatown,40.715618,-73.994279,Möge Tea 愿茶,40.715272,-73.993803,Bubble Tea Shop
1,Chinatown,40.715618,-73.994279,One Cup Tea,40.716893,-73.994596,Bubble Tea Shop
2,Chinatown,40.715618,-73.994279,Kung Fu Tea (功夫茶),40.717119,-73.994562,Bubble Tea Shop
3,Chinatown,40.715618,-73.994279,Coco Fresh Tea & Juice,40.714179,-73.993810,Bubble Tea Shop
4,Chinatown,40.715618,-73.994279,Teado,40.717184,-73.994883,Bubble Tea Shop


In [13]:
newyork_venues_bubbleTeaShop.shape

(639, 7)

In [14]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [15]:
map_newyork_bubbleTeaShop = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_bubbleTeaShop, 'red', map_newyork_bubbleTeaShop)

map_newyork_bubbleTeaShop

In [16]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [17]:
manhattan_grouped = newyork_venues_sushi.groupby('Neighborhood').count()
manhattan_grouped
#print('There are {} uniques categories.'.format(len(newyork_venues_sushi['Venue Category'].unique())))

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,10,10,10,10,10,10
Carnegie Hill,1,1,1,1,1,1
Central Harlem,2,2,2,2,2,2
Chelsea,5,5,5,5,5,5
Chinatown,45,45,45,45,45,45
Civic Center,49,49,49,49,49,49
Clinton,16,16,16,16,16,16
East Village,31,31,31,31,31,31
Financial District,10,10,10,10,10,10


In [18]:
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_bubbleTeaShop[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_bubbleTeaShop['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Asian Restaurant,Bubble Tea Shop,Café,Fried Chicken Joint,Juice Bar,Poke Place,Smoothie Shop,Tea Room,Vietnamese Restaurant
0,Chinatown,0,1,0,0,0,0,0,0,0
1,Chinatown,0,1,0,0,0,0,0,0,0
2,Chinatown,0,1,0,0,0,0,0,0,0
3,Chinatown,0,1,0,0,0,0,0,0,0
4,Chinatown,0,1,0,0,0,0,0,0,0


In [19]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Asian Restaurant,Bubble Tea Shop,Café,Fried Chicken Joint,Juice Bar,Poke Place,Smoothie Shop,Tea Room,Vietnamese Restaurant
0,Battery Park City,0.000000,0.800000,0.000000,0.000000,0.100000,0.00,0.100000,0.000000,0.000000
1,Carnegie Hill,0.000000,1.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2,Central Harlem,0.000000,1.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
3,Chelsea,0.000000,0.800000,0.000000,0.000000,0.000000,0.00,0.000000,0.200000,0.000000
4,Chinatown,0.000000,0.933333,0.044444,0.000000,0.000000,0.00,0.000000,0.022222,0.000000
5,Civic Center,0.000000,0.897959,0.040816,0.000000,0.020408,0.00,0.020408,0.020408,0.000000
6,Clinton,0.000000,0.937500,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.062500
7,East Village,0.000000,0.903226,0.000000,0.032258,0.064516,0.00,0.000000,0.000000,0.000000
8,Financial District,0.000000,0.800000,0.000000,0.000000,0.100000,0.00,0.100000,0.000000,0.000000
9,Flatiron,0.000000,0.950000,0.000000,0.025000,0.000000,0.00,0.000000,0.025000,0.000000


In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 9

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue
0,Battery Park City,Bubble Tea Shop,Smoothie Shop,Juice Bar,Vietnamese Restaurant,Tea Room,Poke Place,Fried Chicken Joint,Café,Asian Restaurant
1,Carnegie Hill,Bubble Tea Shop,Vietnamese Restaurant,Tea Room,Smoothie Shop,Poke Place,Juice Bar,Fried Chicken Joint,Café,Asian Restaurant
2,Central Harlem,Bubble Tea Shop,Vietnamese Restaurant,Tea Room,Smoothie Shop,Poke Place,Juice Bar,Fried Chicken Joint,Café,Asian Restaurant
3,Chelsea,Bubble Tea Shop,Tea Room,Vietnamese Restaurant,Smoothie Shop,Poke Place,Juice Bar,Fried Chicken Joint,Café,Asian Restaurant
4,Chinatown,Bubble Tea Shop,Café,Tea Room,Vietnamese Restaurant,Smoothie Shop,Poke Place,Juice Bar,Fried Chicken Joint,Asian Restaurant


In [28]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 0, 1, 0, 1, 0, 1, 1, 0])

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Manhattan,Chinatown,40.715618,-73.994279,0.0,Bubble Tea Shop,Café,Tea Room,Vietnamese Restaurant,Smoothie Shop,Poke Place,Juice Bar,Fried Chicken Joint,Asian Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,0.0,Bubble Tea Shop,Vietnamese Restaurant,Tea Room,Smoothie Shop,Poke Place,Juice Bar,Fried Chicken Joint,Café,Asian Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0.0,Bubble Tea Shop,Vietnamese Restaurant,Tea Room,Smoothie Shop,Poke Place,Juice Bar,Fried Chicken Joint,Café,Asian Restaurant


In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]



In [37]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [38]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue
1,Chinatown,Bubble Tea Shop,Café,Tea Room,Vietnamese Restaurant,Smoothie Shop,Poke Place,Juice Bar,Fried Chicken Joint,Asian Restaurant
2,Washington Heights,Bubble Tea Shop,Vietnamese Restaurant,Tea Room,Smoothie Shop,Poke Place,Juice Bar,Fried Chicken Joint,Café,Asian Restaurant
4,Hamilton Heights,Bubble Tea Shop,Vietnamese Restaurant,Tea Room,Smoothie Shop,Poke Place,Juice Bar,Fried Chicken Joint,Café,Asian Restaurant
5,Manhattanville,Bubble Tea Shop,Vietnamese Restaurant,Tea Room,Smoothie Shop,Poke Place,Juice Bar,Fried Chicken Joint,Café,Asian Restaurant
6,Central Harlem,Bubble Tea Shop,Vietnamese Restaurant,Tea Room,Smoothie Shop,Poke Place,Juice Bar,Fried Chicken Joint,Café,Asian Restaurant
14,Clinton,Bubble Tea Shop,Vietnamese Restaurant,Tea Room,Smoothie Shop,Poke Place,Juice Bar,Fried Chicken Joint,Café,Asian Restaurant
15,Midtown,Bubble Tea Shop,Vietnamese Restaurant,Tea Room,Smoothie Shop,Poke Place,Juice Bar,Fried Chicken Joint,Café,Asian Restaurant
16,Murray Hill,Bubble Tea Shop,Tea Room,Vietnamese Restaurant,Smoothie Shop,Poke Place,Juice Bar,Fried Chicken Joint,Café,Asian Restaurant
18,Greenwich Village,Bubble Tea Shop,Café,Vietnamese Restaurant,Tea Room,Smoothie Shop,Poke Place,Juice Bar,Fried Chicken Joint,Asian Restaurant
22,Little Italy,Bubble Tea Shop,Café,Tea Room,Vietnamese Restaurant,Smoothie Shop,Poke Place,Juice Bar,Fried Chicken Joint,Asian Restaurant


In [39]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue
8,Upper East Side,Bubble Tea Shop,Café,Vietnamese Restaurant,Tea Room,Smoothie Shop,Poke Place,Juice Bar,Fried Chicken Joint,Asian Restaurant
10,Lenox Hill,Bubble Tea Shop,Café,Asian Restaurant,Vietnamese Restaurant,Tea Room,Smoothie Shop,Poke Place,Juice Bar,Fried Chicken Joint
11,Roosevelt Island,Bubble Tea Shop,Asian Restaurant,Vietnamese Restaurant,Tea Room,Smoothie Shop,Poke Place,Juice Bar,Fried Chicken Joint,Café
13,Lincoln Square,Bubble Tea Shop,Tea Room,Vietnamese Restaurant,Smoothie Shop,Poke Place,Juice Bar,Fried Chicken Joint,Café,Asian Restaurant
17,Chelsea,Bubble Tea Shop,Tea Room,Vietnamese Restaurant,Smoothie Shop,Poke Place,Juice Bar,Fried Chicken Joint,Café,Asian Restaurant
19,East Village,Bubble Tea Shop,Juice Bar,Fried Chicken Joint,Vietnamese Restaurant,Tea Room,Smoothie Shop,Poke Place,Café,Asian Restaurant
20,Lower East Side,Bubble Tea Shop,Juice Bar,Café,Vietnamese Restaurant,Tea Room,Smoothie Shop,Poke Place,Fried Chicken Joint,Asian Restaurant
21,Tribeca,Bubble Tea Shop,Tea Room,Juice Bar,Café,Vietnamese Restaurant,Smoothie Shop,Poke Place,Fried Chicken Joint,Asian Restaurant
27,Gramercy,Bubble Tea Shop,Juice Bar,Tea Room,Fried Chicken Joint,Vietnamese Restaurant,Smoothie Shop,Poke Place,Café,Asian Restaurant
28,Battery Park City,Bubble Tea Shop,Smoothie Shop,Juice Bar,Vietnamese Restaurant,Tea Room,Poke Place,Fried Chicken Joint,Café,Asian Restaurant


In [40]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue
37,Stuyvesant Town,Bubble Tea Shop,Juice Bar,Vietnamese Restaurant,Tea Room,Smoothie Shop,Poke Place,Fried Chicken Joint,Café,Asian Restaurant


In [41]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue
9,Yorkville,Café,Bubble Tea Shop,Vietnamese Restaurant,Tea Room,Smoothie Shop,Poke Place,Juice Bar,Fried Chicken Joint,Asian Restaurant


In [42]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue
12,Upper West Side,Smoothie Shop,Poke Place,Juice Bar,Bubble Tea Shop,Vietnamese Restaurant,Tea Room,Fried Chicken Joint,Café,Asian Restaurant
